In [1]:
import os
import datetime
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
dataset_dir = '/Users/sivaprasanth/Documents/DL/Ex5/Celebrity Faces Dataset'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [4]:
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

Found 1440 images belonging to 17 classes.


In [5]:
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 360 images belonging to 17 classes.


In [6]:
def create_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(17, activation='softmax')
    ])
    return model

In [7]:
model = create_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
class CustomImageLogger(tf.keras.callbacks.Callback):
    def __init__(self, log_dir, validation_data):
        super().__init__()
        self.log_dir = log_dir
        self.validation_data = validation_data
    
    def on_epoch_end(self, epoch, logs=None):
        val_images, val_labels = next(iter(self.validation_data))
        with tf.summary.create_file_writer(self.log_dir).as_default():
            tf.summary.image("Validation Images", val_images, max_outputs=25, step=epoch)

In [9]:
log_dir = "logs/fit_images/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [10]:
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
image_logger_callback = CustomImageLogger(log_dir=log_dir, validation_data=validation_generator)

In [11]:
model.fit(
    x=train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=[tensorboard_callback, image_logger_callback]
)


Epoch 1/5
45/45 [==============================] - 11s 219ms/step - loss: 2.8173 - accuracy: 0.1076 - val_loss: 2.7726 - val_accuracy: 0.1111
Epoch 2/5
45/45 [==============================] - 11s 235ms/step - loss: 2.7269 - accuracy: 0.1312 - val_loss: 2.6302 - val_accuracy: 0.1889
Epoch 3/5
45/45 [==============================] - 9s 194ms/step - loss: 2.4968 - accuracy: 0.2090 - val_loss: 2.3396 - val_accuracy: 0.2333
Epoch 4/5
45/45 [==============================] - 9s 211ms/step - loss: 2.2990 - accuracy: 0.2535 - val_loss: 2.1993 - val_accuracy: 0.2833
Epoch 5/5
45/45 [==============================] - 7s 158ms/step - loss: 2.1213 - accuracy: 0.3181 - val_loss: 2.0762 - val_accuracy: 0.2639


# To view TensorBoard, run this in your terminal:
# tensorboard --logdir=path/to/log-directory